# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f24d819f7f0>
├── 'a' --> tensor([[ 0.0975,  0.7004, -1.9068],
│                   [ 1.8763,  0.0935, -2.1562]])
└── 'x' --> <FastTreeValue 0x7f24d8196940>
    └── 'c' --> tensor([[-2.0255, -0.0900, -1.6164, -1.6285],
                        [ 0.0803,  1.6014, -0.9514, -0.0549],
                        [-0.1968,  2.1201,  2.5206, -2.4505]])

In [4]:
t.a

tensor([[ 0.0975,  0.7004, -1.9068],
        [ 1.8763,  0.0935, -2.1562]])

In [5]:
%timeit t.a

65 ns ± 0.0433 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f24d819f7f0>
├── 'a' --> tensor([[ 1.4579, -0.0298,  0.0335],
│                   [ 1.1801,  0.1775, -1.0357]])
└── 'x' --> <FastTreeValue 0x7f24d8196940>
    └── 'c' --> tensor([[-2.0255, -0.0900, -1.6164, -1.6285],
                        [ 0.0803,  1.6014, -0.9514, -0.0549],
                        [-0.1968,  2.1201,  2.5206, -2.4505]])

In [7]:
%timeit t.a = new_value

61.5 ns ± 0.0317 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.0975,  0.7004, -1.9068],
               [ 1.8763,  0.0935, -2.1562]]),
    x: Batch(
           c: tensor([[-2.0255, -0.0900, -1.6164, -1.6285],
                      [ 0.0803,  1.6014, -0.9514, -0.0549],
                      [-0.1968,  2.1201,  2.5206, -2.4505]]),
       ),
)

In [10]:
b.a

tensor([[ 0.0975,  0.7004, -1.9068],
        [ 1.8763,  0.0935, -2.1562]])

In [11]:
%timeit b.a

54.2 ns ± 0.0461 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.4293, -0.0731,  0.1325],
               [-0.0435, -1.4961, -1.5898]]),
    x: Batch(
           c: tensor([[-2.0255, -0.0900, -1.6164, -1.6285],
                      [ 0.0803,  1.6014, -0.9514, -0.0549],
                      [-0.1968,  2.1201,  2.5206, -2.4505]]),
       ),
)

In [13]:
%timeit b.a = new_value

503 ns ± 0.188 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

828 ns ± 0.611 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.2 µs ± 40.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

137 µs ± 924 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

139 µs ± 419 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f24d820b040>
├── 'a' --> tensor([[[ 0.0975,  0.7004, -1.9068],
│                    [ 1.8763,  0.0935, -2.1562]],
│           
│                   [[ 0.0975,  0.7004, -1.9068],
│                    [ 1.8763,  0.0935, -2.1562]],
│           
│                   [[ 0.0975,  0.7004, -1.9068],
│                    [ 1.8763,  0.0935, -2.1562]],
│           
│                   [[ 0.0975,  0.7004, -1.9068],
│                    [ 1.8763,  0.0935, -2.1562]],
│           
│                   [[ 0.0975,  0.7004, -1.9068],
│                    [ 1.8763,  0.0935, -2.1562]],
│           
│                   [[ 0.0975,  0.7004, -1.9068],
│                    [ 1.8763,  0.0935, -2.1562]],
│           
│                   [[ 0.0975,  0.7004, -1.9068],
│                    [ 1.8763,  0.0935, -2.1562]],
│           
│                   [[ 0.0975,  0.7004, -1.9068],
│                    [ 1.8763,  0.0935, -2.1562]]])
└── 'x' --> <FastTreeValue 0x7f24234a0e80>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33 µs ± 49.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f2423494be0>
├── 'a' --> tensor([[ 0.0975,  0.7004, -1.9068],
│                   [ 1.8763,  0.0935, -2.1562],
│                   [ 0.0975,  0.7004, -1.9068],
│                   [ 1.8763,  0.0935, -2.1562],
│                   [ 0.0975,  0.7004, -1.9068],
│                   [ 1.8763,  0.0935, -2.1562],
│                   [ 0.0975,  0.7004, -1.9068],
│                   [ 1.8763,  0.0935, -2.1562],
│                   [ 0.0975,  0.7004, -1.9068],
│                   [ 1.8763,  0.0935, -2.1562],
│                   [ 0.0975,  0.7004, -1.9068],
│                   [ 1.8763,  0.0935, -2.1562],
│                   [ 0.0975,  0.7004, -1.9068],
│                   [ 1.8763,  0.0935, -2.1562],
│                   [ 0.0975,  0.7004, -1.9068],
│                   [ 1.8763,  0.0935, -2.1562]])
└── 'x' --> <FastTreeValue 0x7f2423b3c580>
    └── 'c' --> tensor([[-2.0255, -0.0900, -1.6164, -1.6285],
                        [ 0.0803,  1.6014, -0.9514, -0.0549],
                 

In [23]:
%timeit t_cat(trees)

30.4 µs ± 61.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

59.1 µs ± 151 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.0975,  0.7004, -1.9068],
                [ 1.8763,  0.0935, -2.1562]],
       
               [[ 0.0975,  0.7004, -1.9068],
                [ 1.8763,  0.0935, -2.1562]],
       
               [[ 0.0975,  0.7004, -1.9068],
                [ 1.8763,  0.0935, -2.1562]],
       
               [[ 0.0975,  0.7004, -1.9068],
                [ 1.8763,  0.0935, -2.1562]],
       
               [[ 0.0975,  0.7004, -1.9068],
                [ 1.8763,  0.0935, -2.1562]],
       
               [[ 0.0975,  0.7004, -1.9068],
                [ 1.8763,  0.0935, -2.1562]],
       
               [[ 0.0975,  0.7004, -1.9068],
                [ 1.8763,  0.0935, -2.1562]],
       
               [[ 0.0975,  0.7004, -1.9068],
                [ 1.8763,  0.0935, -2.1562]]]),
    x: Batch(
           c: tensor([[[-2.0255, -0.0900, -1.6164, -1.6285],
                       [ 0.0803,  1.6014, -0.9514, -0.0549],
                       [-0.1968,  2.1201,  2.5206, -2.4505]],
         

In [26]:
%timeit Batch.stack(batches)

78.5 µs ± 189 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.0975,  0.7004, -1.9068],
               [ 1.8763,  0.0935, -2.1562],
               [ 0.0975,  0.7004, -1.9068],
               [ 1.8763,  0.0935, -2.1562],
               [ 0.0975,  0.7004, -1.9068],
               [ 1.8763,  0.0935, -2.1562],
               [ 0.0975,  0.7004, -1.9068],
               [ 1.8763,  0.0935, -2.1562],
               [ 0.0975,  0.7004, -1.9068],
               [ 1.8763,  0.0935, -2.1562],
               [ 0.0975,  0.7004, -1.9068],
               [ 1.8763,  0.0935, -2.1562],
               [ 0.0975,  0.7004, -1.9068],
               [ 1.8763,  0.0935, -2.1562],
               [ 0.0975,  0.7004, -1.9068],
               [ 1.8763,  0.0935, -2.1562]]),
    x: Batch(
           c: tensor([[-2.0255, -0.0900, -1.6164, -1.6285],
                      [ 0.0803,  1.6014, -0.9514, -0.0549],
                      [-0.1968,  2.1201,  2.5206, -2.4505],
                      [-2.0255, -0.0900, -1.6164, -1.6285],
                      [ 0.0803,  

In [28]:
%timeit Batch.cat(batches)

145 µs ± 377 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

313 µs ± 1.79 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
